In [ ]:
data_path = '../data/'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io.arff import loadarff
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
random_state = 2021

## Year $N$
Run all in this section...

In [ ]:
N = '5'
drop_cols = ['Attr37', 'Attr7','Attr43','Attr62','Attr32','Attr44','Attr15','Attr19','Attr3','Attr51','Attr4','Attr49','Attr38','Attr60','Attr6']

In [ ]:
arff = loadarff(data_path+N+'year.arff')
df = pd.DataFrame(arff[0])
df['class']= df['class'].astype('int')
df = df.drop_duplicates()
# df.head()

In [ ]:
# change dropcols accordingly...
df.isnull().sum().sort_values(ascending=False).head()

In [ ]:
df = df.drop(drop_cols,axis='columns')
# df.shape

### Retain NaN in test set also

In [ ]:
X = df.drop('class',axis='columns')
Y = df['class']
# (X.shape,Y.shape)

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, random_state=random_state,shuffle=True,stratify=Y)

display(Y_train.value_counts(), Y_test.value_counts())

In [ ]:
X_train,Y_train,X_test,Y_test = X_train.to_numpy(),Y_train.to_numpy(),X_test.to_numpy(),Y_test.to_numpy()

## Missing values

In [ ]:
from sklearn.impute import SimpleImputer
simple_imp = SimpleImputer(missing_values=np.nan, strategy='mean')
simple_imp = simple_imp.fit(X_train)

In [ ]:
X_train_imp = simple_imp.transform(X_train)
X_test_imp = simple_imp.transform(X_test)

## PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca = pca.fit(X_train_imp)

X_train_imp = pca.transform(X_train_imp)
X_test_imp = pca.transform(X_test_imp)

## Standardise

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train_imp)

# Various Classification Models

## Simple Impute

In [ ]:
# choose imputer <<comment blocks accordingly>>

#### for simple
scaled_already = False
# X_train_imp = simple_imp.transform(X_train)
# X_test_imp = simple_imp.transform(X_test)


In [ ]:
# !pip3 install imblearn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier # Voting Ensemble for Classification

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
def try_all_classifiers(X_train, X_test, y_train, y_test, classifiers, sampling  = None, scaler=None):
    '''
    do all imputations before passing here...
    Classifier : array of tuples (classifier,scaling required=True/False)
    '''
    accuracy = [0]*len(classifiers)
    f1 = [0]*len(classifiers)
    precision = [0]*len(classifiers)
    recall = [0]*len(classifiers)
    i = 0
    
    model_pipeline = []
    Pipeline([
        ('sampling', SMOTE()),
        ('classification', LogisticRegression())
    ])
    
    if sampling == "SMOTE":
        model_pipeline.append(('sampling', SMOTE(sampling_strategy=0.6,random_state=random_state) ))
        # X_train, y_train = smote.fit_resample(X_train, y_train)
        print("SMOTE")
    if sampling == "RUS":
        model_pipeline.append(('sampling', RandomUnderSampler(sampling_strategy=0.6,random_state=random_state) ))
        # X_train, y_train = rus.fit_resample(X_train, y_train)
        print("RUS")
    if sampling == "SMOTEENN":
        model_pipeline.append(('sampling', SMOTEENN(sampling_strategy=0.6,random_state=random_state) ))
        # X_train, y_train = smoteenn.fit_resample(X_train, y_train)
        print("SMOTEENN")    

    voting_classifs = []
    models_for_voting = [0,3,5,6,7,8]
    voting_weights = [1,0.5,1,0.5,1,1]
    jj =0 
        
    for i in range(len(classifiers)):
        classif = classifiers[i][0]
        pipe_parameters = classifiers[i][2]
        y_pred = []
        print(classifiers_names[i])
        pipeline = Pipeline(model_pipeline+[('classifier',classif)])
        
        if classifiers[i][1] and not scaled_already:
            print("\t- Requires scaling and not scaled. Doing it now...")
            grid = GridSearchCV(pipeline, pipe_parameters, cv=2, scoring="f1",n_jobs=-1,verbose=1)
            #grid = grid.fit(X_train, y_train)
            grid = grid.fit(scaler.transform(X_train), y_train)

            #classif.fit(scaler.transform(X_train), y_train)
            #y_pred = classif.predict(scaler.transform(X_test))
            
            display(grid.best_params_)
            classif = grid.best_estimator_
            y_pred = grid.predict(scaler.transform(X_test))
            #classif.fit(scaler.transform(), )
            #y_pred = classif.predict(scaler.transform(X_test))
            
        else:
            grid = GridSearchCV(pipeline, pipe_parameters, cv=2, scoring="f1",n_jobs=-1,verbose=1)
            grid.fit(X_train, y_train)
            display(grid.best_params_)
            classif = grid.best_estimator_
            y_pred = grid.predict(X_test)
            
            #classif.fit(X_train, y_train)
            #y_pred = classif.predict(X_test)
        
        if i in models_for_voting:
            print("\t- Adding for voting with weight <"+str(voting_weights[jj])+">...")
            jj+=1
            voting_classifs.append(("mod"+str(i+1),classif))
        
        accuracy[i] = metrics.accuracy_score(y_test, y_pred)
        f1[i] = metrics.f1_score(y_test, y_pred, labels=np.unique(y_pred))
        precision[i] = metrics.precision_score(y_test, y_pred)
        recall[i] = metrics.recall_score(y_test, y_pred)
    
    print("\n\nVoting...")
    # create the ensemble model
    ensemble = VotingClassifier(voting_classifs,weights=voting_weights,n_jobs=-1,voting="hard")
    ensemble.fit(scaler.transform(X_train), y_train)
    y_pred = ensemble.predict(scaler.transform(X_test))
    
    accuracy.append(metrics.accuracy_score(y_test, y_pred))
    f1.append(metrics.f1_score(y_test, y_pred, labels=np.unique(y_pred)))
    precision.append(metrics.precision_score(y_test, y_pred))
    recall.append(metrics.recall_score(y_test, y_pred))

    print("Done")
    return accuracy,f1,precision,recall

In [ ]:
# classifiers_voting = [('log',LogisticRegression(max_iter=2048)),("knn10",KNeighborsClassifier(n_neighbors=10)),("dtc",DecisionTreeClassifier()),("svm_linear",SVC(kernel='linear',random_state=random_state)),("rf",RandomForestClassifier(n_estimators=16, n_jobs=8, random_state=random_state)),("xbg",XGBClassifier(use_label_encoder=False))]
# classifiers_voting = [("dtc",DecisionTreeClassifier()),("rf",RandomForestClassifier(n_estimators=64, n_jobs=-1, random_state=random_state)),("xbg",XGBClassifier(use_label_encoder=False))]

classifiers_names = ["LR", "LDA", "KNN-5", "KNN-10", "GNB", "DT", "SVC", "RFC", "XGB","Voting"]

classifiers = [(LogisticRegression(max_iter=2048,random_state=random_state),True,
                   [
                       #{
                       # 'classifier__penalty' : ['l1', 'l2'],
                       # 'classifier__C' : np.logspace(-8, 4, 16),
                       # 'classifier__solver' : ['liblinear']
                       # },
                        {
                        'classifier__penalty' : ['l2','none'],
                        'classifier__C' : np.logspace(-8, 4, 16)
                        }
                   ]),
                (LinearDiscriminantAnalysis(),True,
                    { 'classifier__solver' : ['svd', 'lsqr', 'eigen'] }),
                (KNeighborsClassifier(n_neighbors=5),True,
                    {'classifier__weights' : ['uniform','distance'], 'classifier__metric' : ['euclidean', 'manhattan']}), 
                (KNeighborsClassifier(n_neighbors=10),True,
                    {'classifier__weights' : ['uniform','distance'], 'classifier__metric' : ['euclidean', 'manhattan']}),
                (GaussianNB(),True,
                    {'classifier__var_smoothing': np.logspace(0,-9, num=100)}),
                (DecisionTreeClassifier(random_state=random_state),False,
                    { 'classifier__criterion':['gini','entropy'],'classifier__max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]} ),
                (SVC(random_state=random_state),True,
                    [
                        {'classifier__C': [ 0.05, 0.1, 1], 
                         'classifier__gamma': [0.0001, 1],
                         'classifier__kernel': ['rbf']},
                        {'classifier__C': [ 0.05, 0.1, 1],
                         'classifier__kernel': ['linear']}
                    ]),
                (RandomForestClassifier(random_state=random_state),False,
                     { 'classifier__n_estimators': [int(x) for x in np.linspace(start = 128, stop = 512, num = 4)],
                       'classifier__max_features': ['auto'],
                       'classifier__max_depth':  [int(x) for x in np.linspace(10, 100, num = 2)]+[None],
                       'classifier__min_samples_leaf':  [1, 4],
                       'classifier__bootstrap': [False]
                     }),
                (XGBClassifier(use_label_encoder=False),False,
                    {
                        'classifier__gamma': [0.5, 1, 2, 5],
                        'classifier__colsample_bytree': [0.6, 0.8, 1.0],
                        'classifier__max_depth': [3, 6]
                    }) ]



In [ ]:
accuracy,f1,precision,recall = try_all_classifiers(X_train_imp,X_test_imp,Y_train,Y_test, classifiers, scaler=scaler)


#### Sampling

In [ ]:
accuracy_sm,f1_sm,precision_sm,recall_sm = try_all_classifiers(X_train_imp,X_test_imp,Y_train,Y_test, classifiers, sampling = "SMOTE", scaler=scaler)


In [ ]:
accuracy_rus,f1_rus,precision_rus,recall_rus = try_all_classifiers(X_train_imp,X_test_imp,Y_train,Y_test, classifiers, sampling = "RUS", scaler=scaler)


In [ ]:
accuracy_smoteenn,f1_smoteenn,precision_smoteenn,recall_smoteenn = try_all_classifiers(X_train_imp,X_test_imp,Y_train,Y_test, classifiers, sampling = "SMOTEENN",scaler=scaler)


In [ ]:
print("Imputer & Sampling & Metric & ",end = "")
print(*classifiers_names,sep = " & ", end = " \\\\\n")
print("\\hline \\hline")
print("Simple & No & Acc & ",end="")
print(*['%.2f' % elem for elem in accuracy],sep=" & ", end = " \\\\\n")
print("~ & ~ & Prec & ",end="")
print(*['%.2f' % elem for elem in precision],sep=" & ", end = " \\\\\n")
print("~ & ~ & Rec & ",end="")
print(*['%.2f' % elem for elem in recall],sep=" & ", end = " \\\\\n")
print("~ & ~ & F1 & ",end="")
print(*['%.2f' % elem for elem in f1],sep=" & ", end = " \\\\\n")
print("\cline{2-13}")

print("~ & SMOTE & Acc & ",end="")
print(*['%.2f' % elem for elem in accuracy_sm],sep=" & ", end = " \\\\\n")
print("~ & ~ & Prec & ",end="")
print(*['%.2f' % elem for elem in precision_sm],sep=" & ", end = " \\\\\n")
print("~ & ~ & Rec & ",end="")
print(*['%.2f' % elem for elem in recall_sm],sep=" & ", end = " \\\\\n")
print("~ & ~ & F1 & ",end="")
print(*['%.2f' % elem for elem in f1_sm],sep=" & ", end = " \\\\\n")
print("\cline{2-13}")

print("~ & RUS & Acc & ",end="")
print(*['%.2f' % elem for elem in accuracy_rus],sep=" & ", end = " \\\\\n")
print("~ & ~ & Prec & ",end="")
print(*['%.2f' % elem for elem in precision_rus],sep=" & ", end = " \\\\\n")
print("~ & ~ & Rec & ",end="")
print(*['%.2f' % elem for elem in recall_rus],sep=" & ", end = " \\\\\n")
print("~ & ~ & F1 & ",end="")
print(*['%.2f' % elem for elem in f1_rus],sep=" & ", end = " \\\\\n")
print("\cline{2-13}")


print("~ & SMOTE- & Acc & ",end="")
print(*['%.2f' % elem for elem in accuracy_smoteenn],sep=" & ", end = " \\\\\n")
print("~ & ENN & Prec & ",end="")
print(*['%.2f' % elem for elem in precision_smoteenn],sep=" & ", end = " \\\\\n")
print("~ & ~ & Rec & ",end="")
print(*['%.2f' % elem for elem in recall_smoteenn],sep=" & ", end = " \\\\\n")
print("~ & ~ & F1 & ",end="")
print(*['%.2f' % elem for elem in f1_smoteenn],sep=" & ", end = " \\\\\n")
print("\\hline\\hline")




In [ ]:
print('''
| classifier          | Accuracy | Precision | Recall | F1 score |
| =================== | ======== | ========= | ====== | ======== |''')
for c,a,p,r,f in zip(classifiers_names,accuracy,precision,recall,f1):
    print(c,"  |  ",a,"  |  ",p,"  |  ",r,"  |  ",f,"  |  ")
    
print("\n===============================================================\n")
for c,a,p,r,f in zip(classifiers_names,accuracy_sm,precision_sm,recall_sm,f1_sm):
    print(c,"  |  ",a,"  |  ",p,"  |  ",r,"  |  ",f,"  |  ")

print("\n===============================================================\n")
for c,a,p,r,f in zip(classifiers_names,accuracy_rus,precision_rus,recall_rus,f1_rus):
    print(c,"  |  ",a,"  |  ",p,"  |  ",r,"  |  ",f,"  |  ")
  
print("\n===============================================================\n")
for c,a,p,r,f in zip(classifiers_names,accuracy_smoteenn,precision_smoteenn,recall_smoteenn,f1_smoteenn):
    print(c,"  |  ",a,"  |  ",p,"  |  ",r,"  |  ",f,"  |  ")
 

# ------
# EOF
